# Telco Customer Churn - Model Definition
From the course instructions: model definition depends on your use case and data set how you want to proceed.

Guidelines:
- Choose, justify and apply a model performance indicator (e.g. F1 score, true positive rate, within cluster sum of squared error, …) to assess your model and justify the choice of an algorithm

- Implement your algorithm in at least one deep learning and at least one non-deep learning algorithm, compare and document model performance

- Apply at least one additional iteration in the process model involving at least the feature creation task and record impact on model performance (e.g. data normalizing, PCA, …)

## Model performance indicator
True positive rate (also called recall) and the F1 score will be used to evaluate model performance. Recall is relevant because it would be useful to flag customers showing higher risk of churn. The indicators are given by the following equations:

$recall = \frac{True Positives}{True Positives + False Negatives}$

$precision = \frac{True Positives}{True Positives + False Positives}$

$F_1score = 2*\frac{precision*recall}{precision+recall}$

- Recall is the number of positive predictions divided by the number of positive class values in the test data. It is also called sensitivity, true positive rate, or probability of detection. It measures the proportion of actual positives that are correctly identified as such.
- Precision is the number of positive predictions divided by the total number of positive class values predicted.
- The F1 score conveys the balance between the precision and the recall.

## Import packages

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

Using TensorFlow backend.


## Load data

In [2]:
df = pd.read_csv('../dataset/dataset_model.csv')

In [3]:
df.describe()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,...,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_scaled,MonthlyCharges_scaled,TotalCharges_scaled
count,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,...,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7.032000e+03,7.032000e+03,7.032000e+03
mean,0.504693,0.162400,0.482509,0.298493,32.421786,0.903299,0.592719,64.798208,2283.300441,0.265785,...,0.551052,0.209329,0.239619,0.219283,0.216297,0.336320,0.228100,-1.388410e-16,8.828736e-17,-1.099487e-16
std,0.500014,0.368844,0.499729,0.457629,24.545260,0.295571,0.491363,30.085974,2266.771362,0.441782,...,0.497422,0.406858,0.426881,0.413790,0.411748,0.472483,0.419637,1.000071e+00,1.000071e+00,1.000071e+00
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,18.250000,18.800000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.280248e+00,-1.547283e+00,-9.990692e-01
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,35.587500,401.450000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9.542963e-01,-9.709769e-01,-8.302488e-01
50%,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,70.350000,1397.475000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.394171e-01,1.845440e-01,-3.908151e-01
75%,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,89.862500,3794.737500,1.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,9.199259e-01,8.331482e-01,6.668271e-01
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,118.750000,8684.800000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.612573e+00,1.793381e+00,2.824261e+00


## Data splitting - training, validation, testing
Data will be split into three parts:
- training dataset to obtain model parameters.
- validation dataset to tune hyperparameters and select model.
- testing dataset to evaluate how the model performs on unseen data. It will be used for model selection, but no model tuning.

In [4]:
X = df.drop(['Churn'],axis=1) 
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1234, stratify=y)
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.25, 
                                                            random_state=4321, stratify=y_train)

## Modelling

### Raw numerical features
At a first benchmark models and performance evaluation will be performed on the dataset before scaling/normalizing/centering the numerical features.

In [5]:
# scaled numerical features are removed from the training and validation datasets
X_train_raw = X_train.drop(['tenure_scaled','MonthlyCharges_scaled','TotalCharges_scaled'],axis=1) 
X_validate_raw = X_validate.drop(['tenure_scaled','MonthlyCharges_scaled','TotalCharges_scaled'],axis=1) 

#### Logistic regression
The regression coefficients didn't converge after 1000 interactions. It produced a recall of 0.55 and f1-score of 0.57 for positive churn observations, and an overall weight f1-score of 0.77. 

In [6]:
fit_log_reg = LogisticRegression(solver='saga', max_iter = 1000)
fit_log_reg.fit(X_train_raw, y_train)
y_pred = fit_log_reg.predict(X_validate_raw)

print(confusion_matrix(y_validate, y_pred))
print(classification_report(y_validate, y_pred))

[[942 156]
 [179 218]]
              precision    recall  f1-score   support

           0       0.84      0.86      0.85      1098
           1       0.58      0.55      0.57       397

    accuracy                           0.78      1495
   macro avg       0.71      0.70      0.71      1495
weighted avg       0.77      0.78      0.77      1495



/home/jackson/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


#### Neural networks
The neural network produced a recall of 0.37 and f1-score of 0.49 for positive churn observations, and an overall weight f1-score of 0.77. Please keep in mind that results vary significantly by fitting the model again, due to random initialization of the weights in neural networks.

In [7]:
fit_nn = Sequential()
fit_nn.add(Dense(X_train_raw.shape[1], input_dim = X_train_raw.shape[1], activation='relu'))
#fit_nn.add(Dense(50, activation='relu'))
fit_nn.add(Dense(1, activation='sigmoid'))

fit_nn.compile(optimizer ='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
fit_nn.fit(x=X_train_raw, y=y_train, epochs=50)

Epoch 1/50
4482/4482 [==============================] - 0s 52us/step - loss: 1.0928 - accuracy: 0.7100
Epoch 2/50
4482/4482 [==============================] - 0s 31us/step - loss: 0.4928 - accuracy: 0.7793
Epoch 3/50
4482/4482 [==============================] - 0s 29us/step - loss: 0.6308 - accuracy: 0.7593
Epoch 4/50
4482/4482 [==============================] - 0s 31us/step - loss: 0.6918 - accuracy: 0.7702
Epoch 5/50
4482/4482 [==============================] - 0s 30us/step - loss: 0.6853 - accuracy: 0.7686
Epoch 6/50
4482/4482 [==============================] - 0s 32us/step - loss: 0.5119 - accuracy: 0.7816
Epoch 7/50
4482/4482 [==============================] - 0s 57us/step - loss: 0.6664 - accuracy: 0.7642
Epoch 8/50
4482/4482 [==============================] - 0s 42us/step - loss: 1.1139 - accuracy: 0.7588
Epoch 9/50
4482/4482 [==============================] - 0s 35us/step - loss: 0.5343 - accuracy: 0.7829
Epoch 10/50
4482/4482 [==============================] - 0s 33us/step - l

In [8]:
y_pred = fit_nn.predict_classes(X_validate_raw)
print(confusion_matrix(y_validate, y_pred))
print(classification_report(y_validate, y_pred))

[[1039   59]
 [ 249  148]]
              precision    recall  f1-score   support

           0       0.81      0.95      0.87      1098
           1       0.71      0.37      0.49       397

    accuracy                           0.79      1495
   macro avg       0.76      0.66      0.68      1495
weighted avg       0.78      0.79      0.77      1495



### Scaled numerical features
Based on the results of the previous section, the numerical features will be scaled/normalized/centered moving forward, which should have a positive impact on model performance.

In [9]:
# raw numerical features are removed from the training, validation and test datasets
X_train = X_train.drop(['tenure','MonthlyCharges','TotalCharges'],axis=1) 
X_validate = X_validate.drop(['tenure','MonthlyCharges','TotalCharges'],axis=1) 
X_test = X_test.drop(['tenure','MonthlyCharges','TotalCharges'],axis=1) 

#### Logistic regression
A grid search will be performed to tune the 'C' and 'class_weight' parameters. Recall will be used as scoring method.

In [10]:
fit_log_reg = LogisticRegression(random_state=1234, solver='saga', max_iter = 500)

parameters = {'C':[0.001, 0.1, 1, 100, 10e3], 'class_weight': [None,'balanced']}
grid = GridSearchCV(fit_log_reg, param_grid=parameters, scoring = 'recall', cv=5)

grid.fit(X_train, y_train)
print(grid.best_params_)

{'C': 0.001, 'class_weight': 'balanced'}


In [11]:
y_pred = grid.predict(X_validate)

print(confusion_matrix(y_validate, y_pred))
print(classification_report(y_validate, y_pred))

[[784 314]
 [ 74 323]]
              precision    recall  f1-score   support

           0       0.91      0.71      0.80      1098
           1       0.51      0.81      0.62       397

    accuracy                           0.74      1495
   macro avg       0.71      0.76      0.71      1495
weighted avg       0.81      0.74      0.75      1495



#### Neural networks
The neural network produced a recall of 0.54 and f1-score of 0.59 for positive churn observations, and an overall weight f1-score of 0.80. Please keep in mind that results will vary by fitting the model again, due to random initialization of the weights in neural networks.

In [13]:
fit_nn = Sequential()
fit_nn.add(Dense(X_train.shape[1], input_dim = X_train.shape[1], activation='relu'))
fit_nn.add(Dense(1, activation='sigmoid'))

fit_nn.compile(optimizer ='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
fit_nn.fit(x=X_train, y=y_train, epochs=50)

Epoch 1/50
4482/4482 [==============================] - 0s 49us/step - loss: 0.4721 - accuracy: 0.7731
Epoch 2/50
4482/4482 [==============================] - 0s 31us/step - loss: 0.4244 - accuracy: 0.8008
Epoch 3/50
4482/4482 [==============================] - 0s 30us/step - loss: 0.4187 - accuracy: 0.8041
Epoch 4/50
4482/4482 [==============================] - 0s 31us/step - loss: 0.4153 - accuracy: 0.8068
Epoch 5/50
4482/4482 [==============================] - 0s 31us/step - loss: 0.4124 - accuracy: 0.8077
Epoch 6/50
4482/4482 [==============================] - 0s 33us/step - loss: 0.4113 - accuracy: 0.8070
Epoch 7/50
4482/4482 [==============================] - 0s 30us/step - loss: 0.4092 - accuracy: 0.8104
Epoch 8/50
4482/4482 [==============================] - 0s 30us/step - loss: 0.4082 - accuracy: 0.8097
Epoch 9/50
4482/4482 [==============================] - 0s 32us/step - loss: 0.4067 - accuracy: 0.8110
Epoch 10/50
4482/4482 [==============================] - 0s 32us/step - l

In [14]:
y_pred = fit_nn.predict_classes(X_validate)
print(confusion_matrix(y_validate, y_pred))
print(classification_report(y_validate, y_pred))

[[989 109]
 [184 213]]
              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1098
           1       0.66      0.54      0.59       397

    accuracy                           0.80      1495
   macro avg       0.75      0.72      0.73      1495
weighted avg       0.79      0.80      0.80      1495



#### Neural networks - deep learning
Additional hidden layers were introduced in the model. Dropout was introduced to deal with overfitting the training data. Performance didn't improve significantly compared to the shallow neural network, likely due to the limited amount of data.

In [17]:
fit_nn = Sequential()
fit_nn.add(Dense(X_train.shape[1], input_dim = X_train.shape[1], activation='relu'))
fit_nn.add(Dropout(0.2))
fit_nn.add(Dense(X_train.shape[1], activation='relu'))
fit_nn.add(Dropout(0.2))
fit_nn.add(Dense(20, activation='relu'))
fit_nn.add(Dropout(0.2))
fit_nn.add(Dense(1, activation='sigmoid'))

fit_nn.compile(optimizer ='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
fit_nn.fit(x=X_train, y=y_train, epochs=30)

Epoch 1/30
4482/4482 [==============================] - 0s 98us/step - loss: 0.5116 - accuracy: 0.7419
Epoch 2/30
4482/4482 [==============================] - 0s 49us/step - loss: 0.4441 - accuracy: 0.7916
Epoch 3/30
4482/4482 [==============================] - 0s 74us/step - loss: 0.4377 - accuracy: 0.7921
Epoch 4/30
4482/4482 [==============================] - 0s 64us/step - loss: 0.4358 - accuracy: 0.7972
Epoch 5/30
4482/4482 [==============================] - 0s 49us/step - loss: 0.4303 - accuracy: 0.7954
Epoch 6/30
4482/4482 [==============================] - 0s 45us/step - loss: 0.4247 - accuracy: 0.8012
Epoch 7/30
4482/4482 [==============================] - 0s 46us/step - loss: 0.4263 - accuracy: 0.8023
Epoch 8/30
4482/4482 [==============================] - 0s 49us/step - loss: 0.4215 - accuracy: 0.8077
Epoch 9/30
4482/4482 [==============================] - 0s 49us/step - loss: 0.4199 - accuracy: 0.8095
Epoch 10/30
4482/4482 [==============================] - 0s 45us/step - l

In [15]:
y_pred = fit_nn.predict_classes(X_validate)
print(confusion_matrix(y_validate, y_pred))
print(classification_report(y_validate, y_pred))

[[989 109]
 [184 213]]
              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1098
           1       0.66      0.54      0.59       397

    accuracy                           0.80      1495
   macro avg       0.75      0.72      0.73      1495
weighted avg       0.79      0.80      0.80      1495



### Additional algorithms and tuning of hyperparameters
Additional algorithms will be used on the scaled dataset seeking to evaluate model performance. Hyperparameters will be tuned as well.

#### k-nearest neighbors

In [16]:
fit_knn = KNeighborsClassifier(n_neighbors = 55)

parameters = {'n_neighbors': [43,47,51,55,59]}

grid_knn = GridSearchCV(fit_knn, param_grid=parameters, scoring = 'f1_weighted', cv=5)

grid_knn.fit(X_train, y_train)
grid_knn.best_params_
# {'n_neighbors': 55}

y_pred = grid_knn.predict(X_validate)

print(confusion_matrix(y_validate, y_pred))
print(classification_report(y_validate, y_pred))

[[939 159]
 [151 246]]
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      1098
           1       0.61      0.62      0.61       397

    accuracy                           0.79      1495
   macro avg       0.73      0.74      0.74      1495
weighted avg       0.79      0.79      0.79      1495



#### Support Vector Machines

In [17]:
fit_svm = SVC(C = 100, gamma=0.01, kernel='poly')

#parameters = {'C':[0.001, 0.1, 100, 10e5], 'gamma': [10,1,0.1,0.01]}

#grid = GridSearchCV(fit_svm, param_grid=parameters, scoring = 'recall', cv=5) # caution: it takes a hours to run this grid search

#grid.fit(X_train, y_train)
#grid.best_params_
# {'C': 100, 'gamma': 0.01}

fit_svm.fit(X_train, y_train)
y_pred = fit_svm.predict(X_validate)

print(confusion_matrix(y_validate, y_pred))
print(classification_report(y_validate, y_pred))

[[1015   83]
 [ 196  201]]
              precision    recall  f1-score   support

           0       0.84      0.92      0.88      1098
           1       0.71      0.51      0.59       397

    accuracy                           0.81      1495
   macro avg       0.77      0.72      0.73      1495
weighted avg       0.80      0.81      0.80      1495



#### Gradient Boosting

The code that follows is based on the the blog "Complete Machine Learning Guide to Parameter Tuning in Gradient Boosting (GBM) in Python"

https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

**Tuning of the n_estimators parameter**

In [19]:
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10), 
param_grid = param_test1, scoring = 'recall',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train, y_train)
gsearch1.best_params_, gsearch1.best_score_

/home/jackson/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'n_estimators': 80}, 0.5318272915860904)

**Tuning of the max_depth parameter**

In [20]:
param_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(200,1001,200)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=80, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring = 'recall', n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train, y_train)
gsearch2.best_params_, gsearch2.best_score_

/home/jackson/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'max_depth': 7, 'min_samples_split': 1000}, 0.5511233782215814)

**Tuning of the min_samples_leaf and min_samples_split parameters**

In [21]:
param_test3 = {'min_samples_split':range(100,300,10), 'min_samples_leaf':range(30,71,10)}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=80,max_depth=7,max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test3, scoring = 'recall', n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train, y_train)
gsearch3.best_params_, gsearch3.best_score_

/home/jackson/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'min_samples_leaf': 50, 'min_samples_split': 200}, 0.5410709890650821)

In [22]:
y_pred = gsearch3.predict(X_validate)

print(confusion_matrix(y_validate, y_pred))
print(classification_report(y_validate, y_pred))

[[986 112]
 [168 229]]
              precision    recall  f1-score   support

           0       0.85      0.90      0.88      1098
           1       0.67      0.58      0.62       397

    accuracy                           0.81      1495
   macro avg       0.76      0.74      0.75      1495
weighted avg       0.81      0.81      0.81      1495



## Model evaluation on the validation dataset

The table below summarizes the models that were created and their respective performances on the validation dataset:

| Algorithm | Scaled numerical features | Recall Churn = 1 | f1-score Churn = 1 | weighted f1-score |
|------------------------|------|------|------|------|
| Logistic regression    | No   | 0.55 | 0.57 | 0.77 |
| Neural network         | No   | 0.37 | 0.49 | 0.77 |       
| Logistic regression    | Yes  | 0.81 | 0.62 | 0.75 |       
| Neural network         | Yes  | 0.54 | 0.59 | 0.80 | 
| Deep Neural network    | Yes  | 0.54 | 0.59 | 0.80 |
| k-nearest neighbors    | Yes  | 0.62 | 0.61 | 0.79 |
| Support vector machine | Yes  | 0.51 | 0.59 | 0.80 |
| Gradient boosting      | Yes  | 0.58 | 0.62 | 0.81 |

**Logistic regression** will be retained as the algorithm to create the model in the training step because it showed the best performance, it's simpler and easier to interpret. 